In [19]:
from typing import Literal

import numpy as np
from pprint import pprint

import sklearn
import seaborn as sns

In [20]:
type Sample = list[int | float]
type Data = list[Sample]

type Target = int | float
type Targets = list[Target]

In [21]:
# Activation functions

from abc import ABC, abstractmethod


class ActivationBase(ABC):
    @abstractmethod
    def calc(self, x: Sample) -> list[float]:
        """Apply the activation function to an layer output"""
        pass

#######################################


class ReLU(ActivationBase):
    def calc(self, x) -> list[float]:
        return np.maximum(0, x)


class ELU(ActivationBase):
    def __init__(self, alpha: float = 1.0) -> None:
        self.alpha = alpha
    
    def calc(self, x: Sample) -> list[float]:
        return np.where(x > 0, x, self.alpha * (np.exp(x) - 1))


class Sigmoid(ActivationBase):
    def calc(self, x: Sample) -> list[float]:
        return 1 / (1 + np.exp(-x))


class Tahn(ActivationBase):
    def calc(self, x: Sample) -> list[float]:
        return np.tanh(x)


class Softmax(ActivationBase):
    """returns model 'probabilities' for each class"""

    def calc(self, x: Sample) -> list[float]:
        
        # optimization: make numbers in an array from -inf to 0 because of a np.exp growing
        # and returns an array of floats from 0.0 to 1.0
        max_value = np.max(x)
        x -= max_value

        exp_values = np.exp(x)
        return exp_values / np.sum(exp_values)

In [22]:
# Loss functions


class LossBase(ABC):
    @abstractmethod
    def calc(self, x: Sample, y: Targets) -> float:
        """Apply the loss function to an output layer"""
        pass


class MSELoss(LossBase):
    """For regression"""
    def __init__(self, reduction: Literal['sum', 'mean'] = 'mean') -> None:
        self.reduction = reduction
    
    def calc(self, x: Sample, y: Targets) -> float:

        x = np.array(x)
        loss = (x - y) ** 2

        if self.reduction == 'sum':
            return np.sum(loss)
        elif self.reduction == 'mean':
            return np.mean(loss)


class CrossEntropy(LossBase):
    """For classification"""
    def calc(self, x: Sample, y: Targets) -> float:
        return -np.sum(y * np.log(x))


In [23]:
a = [1, 100, 22, 99]

f = Softmax()

b = f.calc(a)
print(b)
print(sum(b))

[7.39262147e-44 7.31058579e-01 9.74950551e-35 2.68941421e-01]
1.0


In [24]:
class Dataset:
    def __init__(self, data: Data, target_list: Targets) -> None:
        self.data: Data = data
        self._len = len(data)
        self.target_list: Targets = target_list

    def __len__(self) -> int:
        return self._len
    
    def __getitem__(self, index) -> Sample:
        return self.data[index]
    
    def __iter__(self):
        return iter(self.data)


In [86]:
class Linear:
    def __init__(self, n_inputs: int, n_neurons: int, activation: ActivationBase) -> None:
        self.n_inputs = n_inputs
        self.n_neurons = n_neurons
        
        self.weights = self._init_weights()
        self.biases = self._init_biases()
        self.output = []

        self.activation = activation
    
    def _init_weights(self) -> list[float]:
        return np.random.randn(self.n_inputs, self.n_neurons) * 0.1
    
    def _init_biases(self):
        return np.random.randn(self.n_neurons)
    
    def forward(self, inputs) -> None:
        print(f"{inputs = }")
        print(f"{self.weights = }")
        print(f"{self.biases = }")
        output = np.dot(inputs, self.weights)

        print(f"{output = }")

        output += self.biases
        # output = output + self.biases


        self.output = self.activation.calc(output)
        print(f"{self.output = }")


type Layers = list[Linear]

In [105]:
class Model:
    def __init__(self, layers: Layers, loss: LossBase):
        self.layers = layers
        self._layers_len = len(layers)
        self.loss = loss

    def fit(self, dataset: Dataset):
        losses = []

        for i,sample in enumerate(dataset):
            print("_______ Layer", 1, '\n\n')
            self.layers[0].forward(inputs=sample)
            
            for j in range(1, self._layers_len):
                print("______ Layer", j+1, '\n\n')
                self.layers[j].forward(inputs=self.layers[j-1].output)

            targets = dataset.target_list[i]
            
            # Calc loss
            loss = self.calc_loss(targets=targets)
            losses.append(loss)
    
    def predict(self):
        ...
    
    def calc_loss(self, targets: Target) -> float:
        output_layer = self.layers[-1]
        output = output_layer.output

        loss = self.loss.calc(x=output, y=targets)
        return loss

    
    @property
    def weights(self):
        weights = []
        for layer in self.layers:
            weights.append(layer.weights)
        return weights

In [102]:
X_train = [
    # [1,2,3,4],
    [4,3,2,1]
]
y_train = [1,]


X_val = [
    [1,2,3,4],
    # [4,3,2,1]
]
y_val = [1,]


train_dataset = Dataset(data=X_train, target_list=y_train)
# val_dataset = Dataset(data=val_data)

In [106]:
layers = [
    Linear(4,2, activation=ReLU()),
    Linear(2,4, activation=ReLU()),
    Linear(4,1, activation=Softmax()),
]

model = Model(layers=layers, loss=MSELoss())

In [107]:
model.fit(dataset=train_dataset)

_______ Layer 1 


inputs = [4, 3, 2, 1]
self.weights = array([[-0.06112152, -0.12514491],
       [ 0.00027964, -0.00702919],
       [ 0.05637999,  0.11993163],
       [ 0.08326192, -0.07656556]])
self.biases = array([-0.82065462, -0.55801214])
output = array([-0.04762527, -0.35836952])
self.output = array([0., 0.])
______ Layer 2 


inputs = array([0., 0.])
self.weights = array([[ 0.00430376, -0.18055517, -0.21157959,  0.01160333],
       [-0.11759506,  0.11392518, -0.06527293, -0.01540884]])
self.biases = array([ 0.04182254,  0.30385591, -1.04609445,  2.72200013])
output = array([0., 0., 0., 0.])
self.output = array([0.04182254, 0.30385591, 0.        , 2.72200013])
______ Layer 3 


inputs = array([0.04182254, 0.30385591, 0.        , 2.72200013])
self.weights = array([[-0.11583948],
       [-0.07725544],
       [-0.07515247],
       [ 0.1083649 ]])
self.biases = array([-0.09594633])
output = array([0.26665005])
self.output = array([1.])


In [ ]:
model.layers[-1].

In [ ]:
model.weights